In [ ]:
import os
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms.functional as TrF


In [ ]:
batch_size= 16
num_epochs= 10
lr= 1e-3
margin= 2

In [ ]:
class SignatureDataset(Dataset):
    def __init__(self, root_dir, target_file):
        self.root_dir= root_dir
        self.target_file= target_file
    def __getitem__(self, ix):
        img0= os.path.join(self.root_dir, self.target_file.iloc[ix, 0])
        img1= os.path.join(self.root_dir, self.target_file.iloc[ix, 1])
        target= self.target_file.iloc[ix, 2]
        img0, img1= Image.open(img0), Image.open(img1)
        img0, img1= self._transform(img0, img1)
        return img0, img1, target
    def __len__(self):
        return len()
    def _transform(self, img):
        mean, stdev= [0.5, 0.5, 0.5], [0.5, 0.5, 0.5]
        img0, img1= TrF.resize(img, (75, 100), TrF.resize(img, (75, 100))
        img0, img1= TrF.to_tensor(img0), TrF.to_tensor(img1)
        img0, img1= TrF.normalize(img0, mean, stdev), TrF.to_tensor(img1, mean, stdev)
        return img0, img1

In [ ]:
dset= SignatureDataset("data/", "train.csv")
trainloader= DataLoader(dset, batch_size, num_workers= 2)

In [ ]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding,
                 use_act= False, use_norm= False, use_pool= False, dropout= False):
        super(ConvBlock, self).__init__()
        self.use_act= use_act
        self.use_norm= use_norm 
        self.use_pool= use_pool 
        self.dropout= dropout

        self.conv= nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
    def forward(self, x):
        x= self.conv(x)
        if self.use_act:
            x= F.relu(x)
        if self.use_norm:
            x= F.local_response_norm(x, k= 2)
        if self.use_pool:
            x= F.max_pool2d(x, 2, 2)
        if self.use_dropout:
            return F.dropout2d(x, 0.3)
        return x

In [ ]:
class SignNet(nn.Module):
    def __init__(self):
        self.net= nn.Sequential(
            ConvBlock(1, 96, 11, use_act= True, use_norm= True, use_pool= True),
            ConvBlock(96, 256, 5, 1, 2, use_norm= True, use_pool= True, dropout= True),
            ConvBlock(256, 384, 3, 1, 1),
            ConvBlock(384, 256, 3, 1, 1, use_pool= True, dropout= True),
            nn.Flatten(1, -1)
        )
        self.fc1= (18*26*256, 1024)
        self.fc2= (1024, 256)
    def forward(self, x1, x2):
        x1, x2= self.net(x1), self.net(x2)
        x1, x2= self.fc1(x1), self.fc2(x2)
        x1, x2= F.dropout2d(x1), F.dropout2d(x2)
        return self.fc2(x1), self.fc2(x)


In [ ]:
class ContrastiveLoss(nn.Module):
    def __init__(self, margin):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin 

    def forward(self, p1, p2, target):
        d= F.pairwise_distance(op1, op2)
        t1= torch.pow(d, 2)
        t2= torch.pow(torch.clamp(self.margin - d, min=0.0), 2)
        loss= torch.mean((1-target)*t1+ (target)*t2)
        return loss

In [ ]:
model= SignNet().cuda()
loss_fn= ContrastiveLoss(margin)
optimizer= Adam(model.parameters(), lr)

In [ ]:
model.train()
for epoch in range(num_epochs):
    running_loss= 0
    for _, (img1, img2, target) in enumerate(trainloader):
        img1, img2= img1.float().cuda(), img2.float().cuda() 
        target= target.double.cuda()
        optimizer.zero_grad()
        p1, p2= model(img1, img2)
        loss= loss_fn(p1, p2, target)
        running_loss+= loss.item()
        loss.backward()
        optimizer.step()
    print(f"Epoch: {epoch+1}; Loss: {running_loss/len(trainloader)}")

torch.save(model.state_dict(), f"signature_verification.pth")
